In [1]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("alistairking/recyclable-and-household-waste-classification")

print("Path to dataset files:", path)

100%|██████████| 920M/920M [12:17<00:00, 1.31MB/s] 

Extracting files...


Path to dataset files: C:\Users\Caroline Mwanzia\.cache\kagglehub\datasets\alistairking\recyclable-and-household-waste-classification\versions\1


In [3]:
pip install tensorflow

Defaulting to user installation because normal site-packages is not writeable
  Using cached grpcio-1.76.0-cp313-cp313-win_amd64.whl.metadata (3.8 kB)
  Using cached wheel-0.45.1-py3-none-any.whl.metadata (2.3 kB)
   ---------------------------------------- 0.0/332.0 MB ? eta -:--:--
   ---------------------------------------- 0.3/332.0 MB ? eta -:--:--
   ---------------------------------------- 1.0/332.0 MB 5.8 MB/s eta 0:00:57
   ---------------------------------------- 2.1/332.0 MB 4.1 MB/s eta 0:01:20
   ---------------------------------------- 2.6/332.0 MB 4.1 MB/s eta 0:01:21
   ---------------------------------------- 3.1/332.0 MB 3.2 MB/s eta 0:01:43
   ---------------------------------------- 3.9/332.0 MB 3.3 MB/s eta 0:01:39
    --------------------------------------- 4.7/332.0 MB 3.4 MB/s eta 0:01:36
    --------------------------------------- 5.5/332.0 MB 3.5 MB/s eta 0:01:35
    --------------------------------------- 6.3/332.0 MB 3.5 MB/s eta 0:01:35
    ----------------

In [4]:
import tensorflow as tf
import os
import kagglehub # Added import

# Ensure the dataset is downloaded and get its base path
path = kagglehub.dataset_download("alistairking/recyclable-and-household-waste-classification")
dataset_base_path = path # Use the actual returned path

# Correcting the path to the actual class subdirectories
dataset_path = os.path.join(dataset_base_path, 'images', 'images')

train_ds = tf.keras.utils.image_dataset_from_directory(
    dataset_path,
    image_size=(128, 128),
    batch_size=32,
    validation_split=0.2,
    subset="training",
    seed=123
)

val_ds = tf.keras.utils.image_dataset_from_directory(
    dataset_path,
    image_size=(128, 128),
    batch_size=32,
    validation_split=0.2,
    subset="validation",
    seed=123
)

print("Classes:", train_ds.class_names)

Found 15000 files belonging to 30 classes.
Using 12000 files for training.
Found 15000 files belonging to 30 classes.
Using 3000 files for validation.
Classes: ['aerosol_cans', 'aluminum_food_cans', 'aluminum_soda_cans', 'cardboard_boxes', 'cardboard_packaging', 'clothing', 'coffee_grounds', 'disposable_plastic_cutlery', 'eggshells', 'food_waste', 'glass_beverage_bottles', 'glass_cosmetic_containers', 'glass_food_jars', 'magazines', 'newspaper', 'office_paper', 'paper_cups', 'plastic_cup_lids', 'plastic_detergent_bottles', 'plastic_food_containers', 'plastic_shopping_bags', 'plastic_soda_bottles', 'plastic_straws', 'plastic_trash_bags', 'plastic_water_bottles', 'shoes', 'steel_food_cans', 'styrofoam_cups', 'styrofoam_food_containers', 'tea_bags']


In [5]:
import os
print(os.listdir(dataset_path))

['aerosol_cans', 'aluminum_food_cans', 'aluminum_soda_cans', 'cardboard_boxes', 'cardboard_packaging', 'clothing', 'coffee_grounds', 'disposable_plastic_cutlery', 'eggshells', 'food_waste', 'glass_beverage_bottles', 'glass_cosmetic_containers', 'glass_food_jars', 'magazines', 'newspaper', 'office_paper', 'paper_cups', 'plastic_cup_lids', 'plastic_detergent_bottles', 'plastic_food_containers', 'plastic_shopping_bags', 'plastic_soda_bottles', 'plastic_straws', 'plastic_trash_bags', 'plastic_water_bottles', 'shoes', 'steel_food_cans', 'styrofoam_cups', 'styrofoam_food_containers', 'tea_bags']


In [6]:
model = tf.keras.Sequential([
    tf.keras.layers.Rescaling(1./255, input_shape=(128,128,3)),
    tf.keras.layers.Conv2D(16, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Conv2D(32, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(len(train_ds.class_names), activation='softmax')
])

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.fit(train_ds, validation_data=val_ds, epochs=5)


Epoch 1/5


C:\Users\Caroline Mwanzia\AppData\Roaming\Python\Python313\site-packages\keras\src\layers\preprocessing\data_layer.py:95: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


375/375 ━━━━━━━━━━━━━━━━━━━━ 56s 147ms/step - accuracy: 0.1192 - loss: 3.1942 - val_accuracy: 0.2260 - val_loss: 2.8012
Epoch 2/5
375/375 ━━━━━━━━━━━━━━━━━━━━ 46s 51ms/step - accuracy: 0.3349 - loss: 2.3690 - val_accuracy: 0.3883 - val_loss: 2.2744
Epoch 3/5
375/375 ━━━━━━━━━━━━━━━━━━━━ 73s 195ms/step - accuracy: 0.5133 - loss: 1.7353 - val_accuracy: 0.4617 - val_loss: 2.0564
Epoch 4/5
375/375 ━━━━━━━━━━━━━━━━━━━━ 112s 297ms/step - accuracy: 0.6492 - loss: 1.2438 - val_accuracy: 0.5173 - val_loss: 1.9566
Epoch 5/5
375/375 ━━━━━━━━━━━━━━━━━━━━ 20s 54ms/step - accuracy: 0.7503 - loss: 0.8712 - val_accuracy: 0.5650 - val_loss: 1.9875


In [7]:
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

with open("recyclables_model.tflite", "wb") as f:
    f.write(tflite_model)


INFO:tensorflow:Assets written to: C:\Users\CAROLI~1\AppData\Local\Temp\tmpg_eajee9\assets


INFO:tensorflow:Assets written to: C:\Users\CAROLI~1\AppData\Local\Temp\tmpg_eajee9\assets


Saved artifact at 'C:\Users\CAROLI~1\AppData\Local\Temp\tmpg_eajee9'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 128, 128, 3), dtype=tf.float32, name='keras_tensor')
Output Type:
  TensorSpec(shape=(None, 30), dtype=tf.float32, name=None)
Captures:
  2666458787728: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2666458789840: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2666458788496: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2666458789456: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2666458789648: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2666458790224: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2666458790032: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2666458790608: TensorSpec(shape=(), dtype=tf.resource, name=None)


In [8]:
import tensorflow as tf
import os

interpreter = tf.lite.Interpreter(model_path="recyclables_model.tflite")
interpreter.allocate_tensors()

input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

import numpy as np

# Construct path to an example image file from the 'plastic_soda_bottles' class
class_name_to_test = 'plastic_soda_bottles'
original_class_dir_path = os.path.join(dataset_path, class_name_to_test)

# Assuming images are in a 'default' subdirectory within the class directory
image_source_dir = os.path.join(original_class_dir_path, 'default')

# Find the first image file in this directory
# Assuming there are jpg, jpeg, or png files in the directory
image_files = [f for f in os.listdir(image_source_dir) if f.lower().endswith(('.jpg', '.jpeg', '.png'))]
if not image_files:
    raise FileNotFoundError(f"No image files found in {image_source_dir}")

example_image_path = os.path.join(image_source_dir, image_files[0])

img = tf.keras.utils.load_img(example_image_path, target_size=(128,128))
img_array = tf.keras.utils.img_to_array(img)/255.0
img_array = np.expand_dims(img_array, axis=0)

interpreter.set_tensor(input_details[0]['index'], img_array.astype(np.float32))
interpreter.invoke()
output_data = interpreter.get_tensor(output_details[0]['index'])
predicted_class = train_ds.class_names[np.argmax(output_data)]
print("Predicted class:", predicted_class)

Predicted class: plastic_cup_lids


C:\Users\Caroline Mwanzia\AppData\Roaming\Python\Python313\site-packages\tensorflow\lite\python\interpreter.py:457: UserWarning:     Warning: tf.lite.Interpreter is deprecated and is scheduled for deletion in
    TF 2.20. Please use the LiteRT interpreter from the ai_edge_litert package.
    See the [migration guide](https://ai.google.dev/edge/litert/migration)
    for details.
    
  warnings.warn(_INTERPRETER_DELETION_WARNING)


🚀 Benefits of Edge AI in Real-Time Recycling
⚡ Instant decisions: Classify items in milliseconds without cloud delays.

🔒 Privacy: Camera images stay local, not uploaded to external servers.

🌐 Offline capability: Works even without internet (critical for smart bins in rural areas).

🔋 Efficiency: Lightweight models run on Raspberry Pi with low power consumption.

👉 Example: A smart recycling bin with a Pi camera can instantly tell whether an item is plastic, paper, or glass, guiding users to dispose correctly.